In [1]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import numpy as np

In [87]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 1)
        self.conv1 = nn.Conv2d(4, 1, kernel_size=3, stride=1, bias=True)
        
    def save_gradient(self, idx):
        def hook(grad):
            self._grads[idx] = grad
        return hook

    def forward(self, x):
        self._grads = {}
        x.register_hook(self.save_gradient(0))
#         z = self.conv1(x)
        z = self.fc1(x)
        z.register_hook(self.save_gradient(1))
#         x = F.relu(z)
#         x = self.fc2(x)
#         x.register_hook(self.save_gradient(2))
        return z

In [88]:
model = Net().cuda()

In [102]:
bs = 2

In [103]:
x = Variable(torch.from_numpy(np.ones((bs, 320)) * 10), requires_grad=True).cuda().float()

In [104]:
y_hat = model(x)

In [105]:
y_hat.shape

torch.Size([2, 50])

In [106]:
y = Variable(torch.from_numpy(np.ones((bs, 50)) * 2), volatile=True).cuda().float()

In [107]:
model.zero_grad()
_loss = nn.MSELoss()(y_hat, y)
_loss.backward()

In [108]:
model.fc1.weight.grad.view(-1).norm()

Variable containing:
 315.2332
[torch.cuda.FloatTensor of size 1 (GPU 0)]

In [112]:
(model._grads[1][0].pow(2).sum() * x[0].pow(2).sum()).sqrt() * bs

Variable containing:
 315.2332
[torch.cuda.FloatTensor of size 1 (GPU 0)]

In [110]:
model.fc1.bias.grad.norm()

Variable containing:
 1.7622
[torch.cuda.FloatTensor of size 1 (GPU 0)]

In [111]:
(model._grads[1][0].pow(2).sum()).sqrt() * bs

Variable containing:
 1.7622
[torch.cuda.FloatTensor of size 1 (GPU 0)]

In [313]:
model.fc1.weight.grad.norm()

Variable containing:
 13.2514
[torch.cuda.FloatTensor of size 1 (GPU 0)]

In [314]:
model.fc1.bias.grad.norm()

Variable containing:
 0.7408
[torch.cuda.FloatTensor of size 1 (GPU 0)]

In [315]:
model.fc1.weight.grad.norm() + model.fc1.bias.grad.norm()

Variable containing:
 13.9922
[torch.cuda.FloatTensor of size 1 (GPU 0)]

In [295]:
(model._grads[1].pow(2).sum(dim=1) * x.pow(2).sum(dim=1)).sqrt() * bs

Variable containing:
 5.7763
 5.7763
[torch.cuda.FloatTensor of size 2 (GPU 0)]

In [316]:
_x = x[0].unsqueeze(0)
_grads = model._grads[1][0].unsqueeze(0)

In [317]:
torch.mm(torch.mm(torch.mm(_x.t(), _grads), _grads.t()), _x).trace().sqrt() * bs

Variable containing:
 13.2514
[torch.cuda.FloatTensor of size 1 (GPU 0)]

In [318]:
_grads.pow(2).sum().sqrt() * bs

Variable containing:
 0.7408
[torch.cuda.FloatTensor of size 1 (GPU 0)]

In [319]:
model.fc2.weight.grad.norm()

Variable containing:
 5.8010
[torch.cuda.FloatTensor of size 1 (GPU 0)]

In [320]:
model.fc2.bias.grad.norm()

Variable containing:
 1.8708
[torch.cuda.FloatTensor of size 1 (GPU 0)]

In [ ]:
_x = x[0].unsqueeze(0)
_grads = model._grads[2][0].unsqueeze(0)